<a href="https://colab.research.google.com/github/sudhirshahu51/RAG/blob/main/pdf_query_with_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install llama-index langchain chromadb pypdf llama-index-llms-groq llama-index-embeddings-huggingface llama-index-vector-stores-chroma --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 606.2/606.2 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 2.7 MB/s eta 0:00:0

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install llama-index-retrievers-bm25 --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 646.7/646.7 kB 11.6 MB/s eta 0:00:00


In [4]:
# to make async work well with jupyter notebook
import nest_asyncio
nest_asyncio.apply()

# **1. Text Extraction from PDF file**

In [5]:
from llama_index.core import SimpleDirectoryReader

In [6]:
#documents = SimpleDirectoryReader(input_files=["/content/drive/MyDrive/Colab Notebooks/Data/microsoft_annual_report_2022.pdf"]).load_data()sudhirjk
documents = SimpleDirectoryReader(input_files=["/content/drive/MyDrive/Colab Notebooks/Data/microsoft_annual_report_2022.pdf"]).load_data()


In [7]:
# only load pdf files
required_exts = [".pdf"]
path = '/content/drive/MyDrive/Colab Notebooks/Data'
reader = SimpleDirectoryReader(
    input_dir=path,
    required_exts=required_exts,
    recursive=True,
)
documents = reader.load_data()
reader.list_resources()

['/content/drive/MyDrive/Colab Notebooks/Data/LangSmith.pdf',
 '/content/drive/MyDrive/Colab Notebooks/Data/metagpt.pdf',
 '/content/drive/MyDrive/Colab Notebooks/Data/microsoft_annual_report_2022.pdf',
 '/content/drive/MyDrive/Colab Notebooks/Data/mutual_fund_faq.pdf',
 '/content/drive/MyDrive/Colab Notebooks/Data/resume_sudhir_senior_data_scientist.pdf']

In [8]:
len(documents)

154

# **2. Splitting Chunks**

In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, SentenceTransformersTokenTextSplitter
from llama_index.core.node_parser import LangchainNodeParser          #understand sentence and token based splitting can also be done iusing llamaindex or wrapper of langchain in llamaindex.

In [10]:
parser  = LangchainNodeParser(RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", ". ", " ", ""],
    chunk_size=512,
    chunk_overlap=100))
nodes = parser.get_nodes_from_documents(documents)

In [11]:
len(nodes)

1072

# **3. Create embeddings**

In [12]:
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from IPython.display import Markdown, display
import chromadb
import pandas as pd

In [13]:
# define embedding function
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/777 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [14]:
import chromadb
# chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction


In [15]:
chroma_client = chromadb.Client()
chroma_collection = chroma_client.create_collection("microsoft_annual_report_20221")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [16]:
index = VectorStoreIndex(nodes, storage_context=storage_context, embed_model=embed_model, show_progress=True, use_async=True)


Generating embeddings: 100%|██████████| 108/108 [00:15<00:00,  6.78it/s]


In [17]:
#index = VectorStoreIndex.from_documents(
#    documents, storage_context=storage_context, embed_model=embed_model, transformation = [SentenceSplitter(chunk_size=25, chunk_overlap=0),
#        TitleExtractor(),], show_progress=True, use_async=True
#)

# **4. Calling LLM using GROQ API**

In [18]:
from google.colab import userdata
from llama_index.llms.groq import Groq
from llama_index.core.llms import ChatMessage
groq_key = userdata.get('GROQ_API')
llm = Groq(model="llama3-70b-8192", api_key=groq_key) #https://docs.llamaindex.ai/en/stable/examples/llm/groq/#streaming

In [19]:
response = llm.complete("Explain the importance of low latency LLMs")

In [20]:
print(response)

Low-latency Large Language Models (LLMs) are crucial in various applications where real-time or near-real-time processing is essential. Here are some reasons why low-latency LLMs are important:

1. **Interactive Systems**: In interactive systems like chatbots, virtual assistants, and conversational AI, low-latency LLMs enable rapid response times, making the interaction feel more natural and engaging. Delays in response can lead to user frustration and abandonment.
2. **Real-time Decision Making**: In applications like autonomous vehicles, medical diagnosis, or financial trading, low-latency LLMs can quickly process and analyze large amounts of data to make timely decisions. This is critical in situations where every millisecond counts.
3. **Live Streaming and Broadcasting**: Low-latency LLMs can facilitate real-time language translation, sentiment analysis, or content moderation in live streaming and broadcasting applications, enhancing the viewer experience and ensuring timely conten

In [21]:
response = llm.stream_complete("Explain the importance of low latency LLMs")

In [22]:

for r in response:
    print(r.delta, end="")

Low-latency Large Language Models (LLMs) are crucial in various applications where real-time or near-real-time processing is essential. Here are some reasons why low-latency LLMs are important:

1. **Interactive Systems**: In interactive systems like chatbots, virtual assistants, and conversational AI, low-latency LLMs enable rapid response times, making the interaction feel more natural and human-like. This is particularly important in applications where users expect immediate responses, such as customer support or voice assistants.
2. **Real-time Decision Making**: In applications like autonomous vehicles, robotics, or medical diagnosis, low-latency LLMs can process and analyze vast amounts of data in real-time, enabling swift decision-making and reaction to changing circumstances.
3. **Live Streaming and Broadcasting**: Low-latency LLMs can facilitate real-time language translation, captioning, or subtitling for live events, such as conferences, sports, or news broadcasts, ensuring 

# **5. Query Engine**

### **i. BM25 Retriver**

In [23]:
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.retrievers.bm25 import BM25Retriever
import Stemmer

docstore = SimpleDocumentStore()
docstore.add_documents(nodes)

In [24]:
# We can pass in the index, docstore, or list of nodes to create the retriever
bm25_retriever = BM25Retriever.from_defaults(
    docstore=docstore,
    similarity_top_k=2,
    # Optional: We can pass in the stemmer and set the language for stopwords
    # This is important for removing stopwords and stemming the query + text
    # The default is english for both
    stemmer=Stemmer.Stemmer("english"),
    language="english",
)

DEBUG:bm25s:Building index from IDs objects


In [25]:
from llama_index.core.response.notebook_utils import display_source_node

# will retrieve context from specific companies
retrieved_nodes = bm25_retriever.retrieve("Measure revenue sources of mircrosoft?")

for node in retrieved_nodes:
    display_source_node(node, source_length=5000)

**Node ID:** 1f70877c-35db-40bb-beb3-5c2c371b9529<br>**Similarity:** 2.691020965576172<br>**Text:** impracticability of determining the geographic source of the revenue.  
Revenue, classified by significant product and service offerings, was as follows:  
  
(In millions)         
        
Year Ended June 30, 2022 2021 2020 
        
Server products and cloud services $ 67,321  $ 52,589  $ 41,379  
Office products and cloud services  44,862   39,872   35,316  
Windows  24,761   22,488   21,510  
Gaming  16,230   15,370   11,575  
LinkedIn  13,816   10,289   8,077<br>

**Node ID:** edfa3e4b-9fa3-49ea-9f5e-149f1ec1b1e0<br>**Similarity:** 2.681694746017456<br>**Text:** may incur future impairments.  
Equity investments with readily determinable fair values are measured at fair value. Equity investments without readily 
determinable fair values are measured using the equity method or measured at cost with adjustments for observable 
changes in price or impairments (referred to as the measurement alternative). We perform a qualitative assessment on a<br>

### **ii. Vector Store Retriever**

In [26]:
vector_retriever  = index.as_retriever(similarity_top_k=2, )

In [27]:
# Query Data
query_engine = index.as_query_engine(llm = llm)
response = query_engine.query("Measure revenue sources of mircrosoft?")
display(Markdown(f"<b>{response}</b>"))

<b>Microsoft's revenue sources can be measured by the following categories: Server products and cloud services, Office products and cloud services, Windows, Gaming, and LinkedIn. Additionally, product revenue includes sales from operating systems, cross-device productivity applications, server applications, business solution applications, desktop and server management tools, software development tools, video games, and hardware such as PCs, tablets, gaming and entertainment consoles, other intelligent devices, and related accessories.</b>

#**6. Hybrid Search**

In [28]:
from llama_index.core import Settings
Settings.llm = llm
Settings.embed_model = embed_model

## **i. Hybrid Search using Reciprocal Rerank Fusion score.**

In [29]:
from llama_index.core.retrievers import QueryFusionRetriever


retriever = QueryFusionRetriever(
        [vector_retriever,
        bm25_retriever],
        similarity_top_k=5, #Get top 5 nodes from naximum of 2*(5)(vector retriver) + 2*(5)(bm25) = 20 nodes. (5) for 5 artificial queries.
        num_queries = 5,  # 5 Artificial Query generation.
        use_async=True)

In [30]:
# check how decide Alpha for Above

In [31]:
nodes = retriever.retrieve("Measure revenue sources of mircrosoft?")

for node in nodes:
    print(f"Score: {node.score:.2f} - {node.text}...\n-----\n")
    display_source_node(node, source_length=5000)
    print("--------------------------------------------------")


Score: 6.30 - to access our content and services, and the ability to create new experiences through first-party content creators.  
Search and News Advertising  
Our Search and news advertising business is designed to deliver relevant search, native, and display advertising to a 
global audience. We have several partnerships with other companies, including Yahoo, through which we provide and 
monetize search queries. Growth depends on our ability to attract new users, understand intent, and match intent with...
-----



**Node ID:** 009d3e35-1bae-4655-9401-22f385a2e44c<br>**Similarity:** 6.295213222503662<br>**Text:** to access our content and services, and the ability to create new experiences through first-party content creators.  
Search and News Advertising  
Our Search and news advertising business is designed to deliver relevant search, native, and display advertising to a 
global audience. We have several partnerships with other companies, including Yahoo, through which we provide and 
monetize search queries. Growth depends on our ability to attract new users, understand intent, and match intent with<br>

--------------------------------------------------
Score: 4.68 - 84 
REPORT OF INDEPENDENT REGISTERED PUBLIC ACCOUNTING FIRM  
To the Stockholders and the Board of Directors of Microsoft Corporation  
Opinion on the Financial Statements  
We have audited the accompanying consolidated balance sheets of Microsoft Corporation and subsidiaries (the 
“Company”) as of June 30, 2022 and 2021, the related consolidated statements of income, comprehensive income, cash...
-----



**Node ID:** 8b04de8b-d25f-4461-b858-226533de1872<br>**Similarity:** 4.679513454437256<br>**Text:** 84 
REPORT OF INDEPENDENT REGISTERED PUBLIC ACCOUNTING FIRM  
To the Stockholders and the Board of Directors of Microsoft Corporation  
Opinion on the Financial Statements  
We have audited the accompanying consolidated balance sheets of Microsoft Corporation and subsidiaries (the 
“Company”) as of June 30, 2022 and 2021, the related consolidated statements of income, comprehensive income, cash<br>

--------------------------------------------------
Score: 4.44 - Preprint
B A D EMO OF THE EXECUTION
In this section, we outline the complete process of software development using MetaGPT. It begins
with a user’s input command (as shown in Appendix B.1) and ends with software designed according
to the user’s specifications.
B.1 U SER INPUT
Upon receiving an instruction from the user, MetaGPT collaborates with a professional development
team to fulfill the task. Here is a demo of user input:
User input
1 write a python3 GUI app such that you can draw an image with it...
-----



...

--------------------------------------------------
Score: 4.06 - across the Company. 
In addition to our main research and development operations, we also operate Microsoft Research. Microsoft Research is 
one of the world’s largest corporate research organizations and works in close collaboration with top universities around 
the world to advance the state-of-the-art in computer science and a broad range of other disciplines, providing us a unique 
perspective on future trends and contributing to our innovation....
-----



**Node ID:** 19f60756-c424-4fed-9810-26f28040d8bd<br>**Similarity:** 4.064941883087158<br>**Text:** across the Company. 
In addition to our main research and development operations, we also operate Microsoft Research. Microsoft Research is 
one of the world’s largest corporate research organizations and works in close collaboration with top universities around 
the world to advance the state-of-the-art in computer science and a broad range of other disciplines, providing us a unique 
perspective on future trends and contributing to our innovation.<br>

--------------------------------------------------
Score: 3.07 - Office Commercial products and cloud services revenue 
growth 
Revenue from Office Commercial products and cloud 
services (Office 365 subscriptions, the Office 365 portion 
of Microsoft 365 Commercial subscriptions, and Office 
licensed on -premises), comprising Office, Exchange, 
SharePoint, Microsoft Teams, Office 365 Security and 
Compliance, and Microsoft Viva 
    
Office Consumer products and cloud services revenue 
growth 
Revenue from Office Consumer products and cloud...
-----



**Node ID:** bf9fd5a7-5d11-4197-8452-8fd292c3dd04<br>**Similarity:** 3.0657973289489746<br>**Text:** Office Commercial products and cloud services revenue 
growth 
Revenue from Office Commercial products and cloud 
services (Office 365 subscriptions, the Office 365 portion 
of Microsoft 365 Commercial subscriptions, and Office 
licensed on -premises), comprising Office, Exchange, 
SharePoint, Microsoft Teams, Office 365 Security and 
Compliance, and Microsoft Viva 
    
Office Consumer products and cloud services revenue 
growth 
Revenue from Office Consumer products and cloud<br>

--------------------------------------------------


In [32]:
from llama_index.core.query_engine import RetrieverQueryEngine

query_engine = RetrieverQueryEngine(retriever)

In [33]:
response = query_engine.query("Measure revenue sources of mircrosoft?")
display(Markdown(f"<b>{response}</b>"))

<b>Microsoft's revenue sources can be measured through various channels, including Search and News Advertising, which delivers relevant search, native, and display advertising to a global audience. Additionally, the company generates revenue from its Services, which assist customers in developing, deploying, and managing Microsoft server solutions, Microsoft desktop solutions, and Nuance conversational AI and ambient intelligent solutions. Furthermore, Microsoft earns revenue from its OEMs, including regional and local OEMs, as well as system builders, which source Microsoft software for pre-installation and local redistribution. Direct transactions with organizations through Enterprise Agreements and other licensing models also contribute to Microsoft's revenue. Lastly, the company's Office Commercial products and cloud services, including Office 365 subscriptions, Office 365 portion of Microsoft 365 Commercial subscriptions, and Office licensed on-premises, as well as Office Consumer products and cloud services, are significant revenue sources.</b>

## **ii\. Hybrid Search with query expansion and using cosine similarity for final nodes**

# Cross- ReRanker

# **Query Expansion**

In [34]:
!pip install umap-learn --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 4.9 MB/s eta 0:00:00


In [35]:
import umap.umap_ as umap
import numpy as np
from tqdm import tqdm


*   Expansion with multiple queries



In [36]:
def augment_multiple_query(query, llm):

    messages = [
    ChatMessage(
        role="system", content="You are a helpful expert financial research assistant. Your users are asking questions about an annual report. "
            "Suggest up to five additional related questions to help them find the information they need, for the provided question. "
            "Suggest only short questions without compound sentences. Suggest a variety of questions that cover different aspects of the topic."
            "Make sure they are complete questions, and that they are related to the original question."
            "Output one question per line. Do not number the questions."
    ),
    ChatMessage(role="user", content=query ),
    ]
    resp = llm.chat(messages)
    augmented_queries = resp.message.blocks[0].text.split("\n")
    return augmented_queries

In [37]:
original_query = "What were the most important factors that contributed to increases in revenue?"
augmented_queries = augment_multiple_query(original_query, llm)
#print(aug_queries)
display(Markdown(f"{augmented_queries}"))


['What was the total revenue for the year?', 'Which product or service segment generated the most revenue?', 'Were there any significant changes in pricing or sales volume?', 'Did the company experience any notable acquisitions or divestitures?', "Were there any changes in the company's business model or strategy?"]

In [38]:
queries = [original_query] + augmented_queries

In [39]:
import pandas as pd
from typing import Any, List

In [40]:
query = original_query

In [41]:
vector_retriever  = index.as_retriever(similarity_top_k=2,)
bm25_retriever = BM25Retriever.from_defaults(
                      docstore=docstore,
                      similarity_top_k=2,
                      # Optional: We can pass in the stemmer and set the language for stopwords
                      # This is important for removing stopwords and stemming the query + text
                      # The default is english for both
                      stemmer=Stemmer.Stemmer("english"),
                      language="english",
                        )

DEBUG:bm25s:Building index from IDs objects


In [42]:
def hybrid_search(queries = queries,
                  vector_retriever = vector_retriever,
                  bm25_retriever = vector_retriever) -> pd.DataFrame:
    '''Returns unique nodes from hybrid search'''

    all_nodes = []
    for query in queries:
      nodes_v = vector_retriever.retrieve(query)
      nodes_b = bm25_retriever.retrieve(query)
      all_nodes = all_nodes + nodes_v + nodes_b

    #Making list of unique nodes.
    unique_nodes = {}
    for nod in all_nodes:
      if nod.node_id not in unique_nodes.keys():unique_nodes[nod.node_id] = nod.text

    node_df = pd.DataFrame({'node_id': unique_nodes.keys(), 'text': unique_nodes.values()}, index = range(len(unique_nodes)))
    return node_df

In [43]:
node_df = hybrid_search(queries = queries,
                  vector_retriever = vector_retriever,
                  bm25_retriever = vector_retriever)

In [44]:
node_df

,node_id,text
0,5b414024-663e-47f8-a9f8-853d365f7a04,• Windows revenue increased $2.3 billion or 1...
1,4743c01c-2c3e-4d93-ba09-ebc684765ea3,Cost of revenue increased $10.4 billion or 20%...
2,1f70877c-35db-40bb-beb3-5c2c371b9529,impracticability of determining the geographic...
3,7c4f8e35-d544-4bad-b5ee-f7fff47b9569,"fiscal years 2022, 2021, or 2020. Revenue, cla..."
4,42ed05b2-537c-4401-9469-9e9a3080bf88,Revenue \nProduct Revenue and Service and Oth...
5,fa5ad633-3e8c-4f2f-bb35-3b8a14342c67,quarter revenue is driven by corporate year -e...
6,ba8a25e6-d3e7-4a3f-95e8-9f52a4eb4f10,"• Customer mix between consumer, small and med..."
7,77d69e18-ead1-4bf7-b45f-ea3e1cc70c5e,commercial sales. \nKey changes in operating ...
8,014bc008-4020-43ff-bded-c3169b230de5,"healthcare and enterprise AI experience, and t..."
9,55b070d6-9105-4423-8066-05e83171c525,accelerate the growth in our gaming business a...


In [45]:
# Query Data
query_engine = index.as_query_engine(llm = llm)
response = query_engine.query("Measure revenue sources of mircrosoft?")
display(Markdown(f"<b>{response}</b>"))

<b>Microsoft's revenue sources can be measured by the following categories: Server products and cloud services, Office products and cloud services, Windows, Gaming, and LinkedIn. Additionally, Product revenue includes sales from operating systems, cross-device productivity applications, server applications, business solution applications, desktop and server management tools, software development tools, video games, and hardware such as PCs, tablets, gaming and entertainment consoles, other intelligent devices, and related accessories.</b>

1. Take 20 chunks using vector retriver for augumneted queries.
2. Take 20 chunks using BM25 retriver for augumneted queries.
3. remove the duplicates.
4.  Based u[on dot matrix product with query, choose top 5 and pass as context.

Now integrate re-ranker, graph search too and later on image search.  

In [46]:
node_df

,node_id,text
0,5b414024-663e-47f8-a9f8-853d365f7a04,• Windows revenue increased $2.3 billion or 1...
1,4743c01c-2c3e-4d93-ba09-ebc684765ea3,Cost of revenue increased $10.4 billion or 20%...
2,1f70877c-35db-40bb-beb3-5c2c371b9529,impracticability of determining the geographic...
3,7c4f8e35-d544-4bad-b5ee-f7fff47b9569,"fiscal years 2022, 2021, or 2020. Revenue, cla..."
4,42ed05b2-537c-4401-9469-9e9a3080bf88,Revenue \nProduct Revenue and Service and Oth...
5,fa5ad633-3e8c-4f2f-bb35-3b8a14342c67,quarter revenue is driven by corporate year -e...
6,ba8a25e6-d3e7-4a3f-95e8-9f52a4eb4f10,"• Customer mix between consumer, small and med..."
7,77d69e18-ead1-4bf7-b45f-ea3e1cc70c5e,commercial sales. \nKey changes in operating ...
8,014bc008-4020-43ff-bded-c3169b230de5,"healthcare and enterprise AI experience, and t..."
9,55b070d6-9105-4423-8066-05e83171c525,accelerate the growth in our gaming business a...


# **6. Re-ranking**

In [47]:
from sentence_transformers import CrossEncoder
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [48]:
def reranker_func(query:str,  node_df: pd.DataFrame, n = 5,
                  cross_encoder =  cross_encoder) -> List[str]:
    '''Returns top n chunks based on reranking'''
    node_df['original_query'] = original_query
    pairs = node_df[['original_query', 'text']].values.tolist()
    scores = cross_encoder.predict(pairs) #learn how cross encoders works
    node_df['scores'] = scores
    top_chunks = node_df.nlargest(n, ['scores'])['text'].tolist()

    return top_chunks

In [49]:
top_chunks = reranker_func(query = original_query, node_df= node_df)

In [50]:
top_chunks

['• Windows revenue increased $2.3  billion or 10% driven by growth in Windows OEM and Windows \nCommercial. Windows OEM revenue increased 11% driven by continued strength in the commercial PC \nmarket, which has higher revenue per license. Windows Commercial products and cloud services revenue \nincreased 11% driven by demand for Microsoft 365.',
 'Cost of revenue increased $10.4 billion or 20% driven by growth in Microsoft Cloud.  \nGross margin increased $19.8 billion or 17% driven by growth across each of our segments.  \n• Gross margin percentage decreased slightly. Excluding the impact of the fiscal year 2021 change in \naccounting estimate for the useful lives of our server and network equipment, gross margin percentage \nincreased 1 point driven by improvement in Productivity and Business Processes.',
 'commercial sales.  \nKey changes in operating expenses were:  \n• Research and development expenses increased $3.8  billion or 18% driven by investments in cloud \nengineering, 

Call LLM and use prompt template to pass conetxt and get the response and Also try reranker from llamindex directly and call llm using that.

In [51]:
def rag(query,  retrieved_documents, llm, stream: bool = False):
    information = "\n\n".join(retrieved_documents)

    messages = [
    ChatMessage(
        role="system", content="You are a helpful expert financial research assistant. Your users are asking questions about information contained in an annual report. You will be shown the user's question, and the relevant information from the annual report. Answer the user's question using only this information."
    ),
    ChatMessage(role="user", content=f"Question: {query}. \n Information: {information}"),
    ]

    if stream:
      resp = llm.stream_chat(messages)
    else:
      resp = llm.chat(messages)
    return resp


In [52]:
output = rag(query=original_query, retrieved_documents=top_chunks, llm = llm, stream = True)
#for streaming
for r in output:
    print(r.delta, end="")

Based on the information provided, the most important factors that contributed to increases in revenue were:

* Growth in Windows OEM revenue, driven by strength in the commercial PC market, which has higher revenue per license.
* Growth in Windows Commercial products and cloud services revenue, driven by demand for Microsoft 365.

These factors contributed to a 10% increase in Windows revenue, which was a key driver of overall revenue growth.

Now Use Evaluation and then Integrate Graph Search.

# **LLM Evaluation**

In [53]:
original_query = 'How mutual fund performance is calculated?'

In [54]:
augmented_queries = augment_multiple_query(original_query, llm)
display(Markdown(f"{augmented_queries}"))

["What is the fund's investment objective?", 'What are the benchmarks used to evaluate performance?', "How does the fund's performance compare to its peers?", 'What are the fees and expenses associated with the fund?', "What is the fund's risk-adjusted return?"]

In [55]:
queries = [original_query] + augmented_queries

In [56]:
node_df = hybrid_search(queries, vector_retriever, bm25_retriever)
top_chunks = reranker_func(query = original_query, node_df= node_df)

In [57]:
len(top_chunks)

5

In [58]:
output = rag(query=original_query, retrieved_documents=top_chunks, llm = llm)
#print(output)
display(Markdown(f"{output}"))

assistant: According to the provided information, the performance of a mutual fund scheme is calculated and reflected in its Net Asset Value (NAV), which is disclosed on a daily basis. The NAVs are published on the websites of mutual funds and the Association of Mutual Funds in India (AMFI). Additionally, mutual funds are required to publish their performance in the form of half-yearly results, which include returns over a period of time (e.g., last six months, 1 year, 3 years, 5 years, and since inception of schemes). This information allows investors to compare the performance of their schemes with those of other mutual funds under the same category and with benchmarks like BSE Sensitive Index, Nifty, etc.

In [59]:
output.message.blocks[0].text

'According to the provided information, the performance of a mutual fund scheme is calculated and reflected in its Net Asset Value (NAV), which is disclosed on a daily basis. The NAVs are published on the websites of mutual funds and the Association of Mutual Funds in India (AMFI). Additionally, mutual funds are required to publish their performance in the form of half-yearly results, which include returns over a period of time (e.g., last six months, 1 year, 3 years, 5 years, and since inception of schemes). This information allows investors to compare the performance of their schemes with those of other mutual funds under the same category and with benchmarks like BSE Sensitive Index, Nifty, etc.'

In [60]:
!pip install ragas --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.0/176.0 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 61.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not curren

In [61]:
#from ragas.testset.synthesizers.generate import TestsetGenerator

[Link](https://github.com/explodinggradients/ragas/blob/main/docs/howtos/integrations/llamaindex.ipynb)

In [62]:
from ragas.testset import TestsetGenerator

In [63]:
generator = TestsetGenerator.from_llama_index(
    llm=llm,
    embedding_model=embed_model,
)

In [64]:
from langchain_core.documents import Document as LCDocument

Do the the evealution part next week from above link after getting new version relaease from Ragas or do the correction after as [mentioned](https://github.com/explodinggradients/ragas/issues/1695) in library file if installed all in local for deployment testing. Till Then better try Truera in free time.

In [65]:
#generate testset
testset = generator.generate_with_llamaindex_docs(
    documents,
    testset_size=5,
)

NameError: name 'LCDocument' is not defined

In [ ]:
df = testset.to_pandas()
df.head()

In [ ]:
#Lets try an sample question from the generated testset to see if it is working

# **7. Apply Embedding Adaptors to improve the results more.**
from 5th tutorial.

# **8. Use Graph Search**

[link](https://superlinked.com/vectorhub/articles/optimizing-rag-with-hybrid-search-reranking)

# **9. Interaction with Tabular Data and Table extraction using Miner U.**